In [ ]:
import pandas as pd
import os

dfs = {}
for file in os.listdir("vectorized_data"):
    if file.endswith(".csv"):
        df = pd.read_csv("vectorized_data/" + file, index_col=0)
        dfs[file] = df

dfs_no_labels = {}
for key in dfs:
    dfs_no_labels[key] = dfs[key].drop(columns=["category1"])
    
    dfs_no_labels[key].info()

In [ ]:
def calculateMeanAbsCorrelation(df):
    return df.corr().abs().mean()


for key,item in dfs_no_labels.items():
    print(key)
    print(calculateMeanAbsCorrelation(item))
    print("\n")